# Classification with Naive Bayes

In [1]:
import pandas as pd

## The watermelon dataset

In the following we use the Watermelon dataset from Zhou's textbook, where each record represents a watermelon with its features (color, sound, texture, etc) and with a boolean value indicating if the watermelon is ripe or not.
Notice:
Input features have different types: *color, root, sound, texture, umbilicus, surface* are categorical (have a finite number of nominal values that can only enumerated and compared by equality operators), while *density, sugar* are numerical (continuous) and mathematical operators can be used to transform their values.

We can easily read this CSV dataset using pandas.

In [3]:
dataset = pd.read_csv('datasets/watermelon-dataset.csv')
dataset

,ID,color,root,sound,texture,umbilicus,surface,density,sugar,ripe
0,1,green,curly,muffled,clear,hollow,hard,0.697,0.460,True
1,2,dark,curly,dull,clear,hollow,hard,0.774,0.376,True
2,3,dark,curly,muffled,clear,hollow,hard,0.634,0.264,True
3,4,green,curly,dull,clear,hollow,hard,0.608,0.318,True
4,5,light,curly,muffled,clear,hollow,hard,0.556,0.215,True
5,6,green,slightly curly,muffled,clear,slightly hollow,soft,0.403,0.237,True
6,7,dark,slightly curly,muffled,slightly blurry,slightly hollow,soft,0.481,0.149,True
7,8,dark,slightly curly,muffled,clear,slightly hollow,hard,0.437,0.211,True
8,9,dark,slightly curly,dull,slightly blurry,slightly hollow,hard,0.666,0.091,False
9,10,green,straight,crisp,clear,flat,soft,0.243,0.267,False


# Your work: what you have to do

- 1. Use `CategoricalNB` and `GaussianNB` from `scikit-learn` to build a Naive Bayes classifier on the watermelon dataset. For the estimated probabilities, consider the Laplace correction techniques.

- 2. Using the previously trained model, predict the ripe/unripe class of the test example proposed by the book:
*ID=T1, color=green, root=curly, sound=muffled, texture=clear, umbilicous=hollow, surface=hard, density=0.697, sugar=0.460. *

- 3. Giving the dataset size, consider now an hold-out technique in which you draw randomly a single test example and you train the model on the remaining examples.

- 4. Build the confusion matrix and the ROC curves for naive Bayes and decision Trees

- 5. Implement statistical tests (e.g. `ttest_rel` in [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) to compare model trained at 3. to a Decision Tree on the same dataset with an hold-out method. Have the observed differences in predictive performance statistical  significance?

- 1. Use `CategoricalNB` and `GaussianNB` from `scikit-learn` to build a Naive Bayes classifier on the watermelon dataset. For the estimated probabilities, consider the Laplace correction techniques.
- 2. Using the previously trained model, predict the ripe/unripe class of the test example proposed by the book: *ID=T1, color=green, root=curly, sound=muffled, texture=clear, umbilicous=hollow, surface=hard, density=0.697, sugar=0.460. *

In [71]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# ===============================
# 1. Preprocessing separo feature e target
# ===============================
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(exclude=['object']).columns

# Codifica delle categoriche
encoder = OrdinalEncoder()
X_cat = encoder.fit_transform(X[categorical_cols])

# Estraggo anche le numeriche
X_num = X[numeric_cols].to_numpy()

# Codifico il target
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(y)

# ===============================
# 2. Training separato
# ===============================
cat_nb = CategoricalNB(alpha=1, fit_prior=False)
cat_nb.fit(X_cat, y_encoded)

gauss_nb = GaussianNB(var_smoothing=1e-9)
gauss_nb.fit(X_num, y_encoded)

test_example = pd.DataFrame({
    'color': ['green'],
    'root': ['curly'],
    'sound': ['muffled'],
    'texture': ['clear'],
    'umbilicus': ['hollow'],
    'surface': ['hard'],
    'density': [0.697],
    'sugar': [0.460]
})

test_cat = encoder.transform(test_example[categorical_cols])

print(test_cat)
cat_probs = cat_nb.predict_proba(test_cat)
num_probs = gauss_nb.predict_proba(test_example[numeric_cols].to_numpy())

combined_log_probs = np.log(cat_probs) + np.log(num_probs)
y_pred_combined = np.argmax(combined_log_probs, axis=1)



[[1. 0. 2. 1. 1. 0.]]


In [26]:
import numpy as np
import pandas as pd

"""print("=== HYBRID (Categorical + Gaussian) Naive Bayes Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_combined))"""
# 🔹 Nuovo esempio da predire
test_example = pd.DataFrame({
    'color': ['green'],
    'root': ['curly'],
    'sound': ['muffled'],
    'texture': ['clear'],
    'umbilicus': ['hollow'],
    'surface': ['hard'],
    'density': [0.697],
    'sugar': [0.460]
})

# 🔹 Trasformiamo le categoriche con lo stesso encoder del training
test_example[categorical_cols] = encoder.transform(test_example[categorical_cols])

# Ora possiamo predire
y_pred_test = gauss_nb.predict(test_example)

# 🔹 Decodifichiamo la classe numerica nel nome originale
predicted_class = label_enc.inverse_transform(y_pred_test)

print("🔮 Predizione per il test example T1:")
print(predicted_class[0])


🔮 Predizione per il test example T1:
True


- 3. Giving the dataset size, consider now an hold-out technique in which you draw randomly a single test example and you train the model on the remaining examples.

In [29]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import CategoricalNB, GaussianNB

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# Separo le colonne numeriche e categoriche
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(exclude=['object']).columns
encoder = OrdinalEncoder()

# Se ci sono categoriche, le convertiamo in numeri
if len(categorical_cols) > 0:
    X[categorical_cols] = encoder.fit_transform(X[categorical_cols])

#Codifico il target
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(y)

kf = KFold(n_splits=len(X), shuffle=True, random_state=42)

kf.get_n_splits(X)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Modello
    gauss_nb = GaussianNB(var_smoothing=1e-9)

    # Addestramento
    gauss_nb.fit(X_train, y_train)

    # Predizione
    y_pred_gauss = gauss_nb.predict(X_test)

    # Accuracy
    print("=== GaussianNB Results ===")
    print("Accuracy:", accuracy_score(y_test, y_pred_gauss))


KeyError: "None of [Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], dtype='int64')] are in the [columns]"